In [10]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm


In [ ]:
# 1. Carga y preparación de datos
# Leemos los archivos CSV
math_df = pd.read_csv('wb_pisa_math.csv')
reading_df = pd.read_csv('wb_pisa_reading.csv')
science_df = pd.read_csv('wb_pisa_science.csv')

# Función para transformar datos de formato wide a long
def reshape_pisa_data(df, subject):
    # Seleccionamos columnas de años y metadata
    year_cols = [col for col in df.columns if col.endswith('_Mean')]
    years = [int(col.split('_')[0]) for col in year_cols]
    
    # Creamos id_vars con columnas que no son de años
    id_vars = ['Country', 'Treatment', 'Pandemic_Deaths_2020', 
               'Education_Spending_Percent_GDP', 'GDP_Per_Capita_2018',
               'COVID_Death_Percent_Population']
    
    # Reshape a formato long
    df_long = pd.melt(df, 
                      id_vars=id_vars,
                      value_vars=year_cols,
                      var_name='year',
                      value_name=f'{subject}_score')
    
    # Limpiamos la columna año
    df_long['year'] = df_long['year'].str.extract('(\d{4})').astype(int)
    
    return df_long

# Aplicamos la transformación a cada materia
math_long = reshape_pisa_data(math_df, 'math')
reading_long = reshape_pisa_data(reading_df, 'reading')
science_long = reshape_pisa_data(science_df, 'science')

# Combinamos los datasets
pisa_long = math_long.merge(
    reading_long[['Country', 'year', 'reading_score']], 
    on=['Country', 'year']
).merge(
    science_long[['Country', 'year', 'science_score']], 
    on=['Country', 'year']
)


In [ ]:

# 2. Preparación para el PSM (usando datos de 2018)
def prepare_matching_data(df):
    # Filtramos datos de 2018
    matching_data = df[df['year'] == 2018].copy()
    
    # Seleccionamos variables para el matching
    matching_vars = ['GDP_Per_Capita_2018', 'Education_Spending_Percent_GDP',
                    'math_score', 'reading_score', 'science_score']
    
    # Estandarizamos las variables
    scaler = StandardScaler()
    matching_data[matching_vars] = scaler.fit_transform(matching_data[matching_vars])
    
    return matching_data

matching_data = prepare_matching_data(pisa_long)

# 3. Estimación del Propensity Score
def estimate_propensity_score(data):
    # Preparamos X y y para la regresión logística
    X = data[['GDP_Per_Capita_2018', 'Education_Spending_Percent_GDP',
              'math_score', 'reading_score', 'science_score']]
    y = data['Treatment']
    
    # Estimamos el modelo
    pscore_model = LogisticRegression(random_state=42)
    pscore_model.fit(X, y)
    
    # Calculamos los propensity scores
    pscores = pscore_model.predict_proba(X)[:, 1]
    
    return pscores

matching_data['pscore'] = estimate_propensity_score(matching_data)

# 4. Matching
def nearest_neighbor_matching(data, n_neighbors=1):
    treated = data[data['Treatment'] == 1]
    control = data[data['Treatment'] == 0]
    
    matches = pd.DataFrame()
    
    for idx, treated_unit in treated.iterrows():
        # Calculamos distancias a todas las unidades de control
        distances = abs(control['pscore'] - treated_unit['pscore'])
        
        # Encontramos los n vecinos más cercanos
        nearest_neighbors = distances.nsmallest(n_neighbors)
        
        # Agregamos los matches al DataFrame
        for _, control_idx in nearest_neighbors.items():
            match_pair = pd.DataFrame({
                'treated_country': treated_unit['Country'],
                'control_country': control.loc[control_idx, 'Country'],
                'treated_pscore': treated_unit['pscore'],
                'control_pscore': control.loc[control_idx, 'pscore'],
                'distance': distances[control_idx]
            }, index=[0])
            
            matches = pd.concat([matches, match_pair], ignore_index=True)
    
    return matches

matches = nearest_neighbor_matching(matching_data)

# 5. Análisis DiD en la muestra pareada
def did_analysis(data, matches):
    # Creamos indicador de post-tratamiento (2022)
    data['Post'] = (data['year'] >= 2022).astype(int)
    
    # Filtramos solo países pareados
    matched_countries = list(matches['treated_country']) + list(matches['control_country'])
    matched_data = data[data['Country'].isin(matched_countries)].copy()
    
    # Realizamos el análisis para cada materia
    subjects = ['math_score', 'reading_score', 'science_score']
    results = {}
    
    for subject in subjects:
        # Preparamos el modelo
        Y = matched_data[subject]
        X = pd.get_dummies(matched_data[['Treatment', 'Post', 'year', 'Country']], 
                          columns=['year', 'Country'])
        X['Treatment_Post'] = X['Treatment'] * X['Post']
        
        # Estimamos el modelo
        model = sm.OLS(Y, X)
        results[subject] = model.fit(cov_type='cluster', 
                                   cov_kwds={'groups': matched_data['Country']})
    
    return results

did_results = did_analysis(pisa_long, matches)

# 6. Visualización de resultados
def plot_trends(data, matches):
    # Filtramos países pareados
    matched_countries = list(matches['treated_country']) + list(matches['control_country'])
    matched_data = data[data['Country'].isin(matched_countries)].copy()
    
    # Calculamos promedios por grupo y año
    grouped_data = matched_data.groupby(['year', 'Treatment'])[
        ['math_score', 'reading_score', 'science_score']].mean().reset_index()
    
    # Creamos el gráfico
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    subjects = ['math_score', 'reading_score', 'science_score']
    
    for ax, subject in zip(axes, subjects):
        sns.lineplot(data=grouped_data, x='year', y=subject, 
                    hue='Treatment', ax=ax)
        ax.axvline(x=2020, color='red', linestyle='--')
        ax.set_title(subject.replace('_', ' ').title())
    
    plt.tight_layout()
    plt.show()

plot_trends(pisa_long, matches)

# 7. Análisis de robustez
def balance_test(data, matches):
    # Filtramos países pareados
    matched_countries = list(matches['treated_country']) + list(matches['control_country'])
    matched_data = data[data['Country'].isin(matched_countries)].copy()
    
    # Variables para el test de balance
    balance_vars = ['GDP_Per_Capita_2018', 'Education_Spending_Percent_GDP',
                   'math_score', 'reading_score', 'science_score']
    
    # Realizamos t-tests
    balance_results = pd.DataFrame(columns=['Variable', 'Diff', 'P-value'])
    
    for var in balance_vars:
        t_stat, p_val = stats.ttest_ind(
            matched_data[matched_data['Treatment']==1][var],
            matched_data[matched_data['Treatment']==0][var]
        )
        
        balance_results = pd.concat([balance_results,
            pd.DataFrame({
                'Variable': [var],
                'Diff': [t_stat],
                'P-value': [p_val]
            })
        ])
    
    return balance_results

balance_results = balance_test(matching_data, matches)

# Imprimimos resultados
print("\nBalance Test Results:")
print(balance_results)

print("\nDiD Results:")
for subject, result in did_results.items():
    print(f"\n{subject}")
    print(result.summary().tables[1])